<a href="https://colab.research.google.com/github/marlapinkert/final_project_compcognition/blob/main/download_and_correlation_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test - Download script

In [175]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations
from glob import glob

In [6]:
!python download_abide_preproc.py -d rois_cc200 -p cpac -s filt_global -o /content

No upper age threshold specified
No lower age threshold specified
No site specified, using all sites...
No sex specified, using all sexes...
b',Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,VIQ,PIQ,FIQ_TEST_TYPE,VIQ_TEST_TYPE,PIQ_TEST_TYPE,ADI_R_SOCIAL_TOTAL_A,ADI_R_VERBAL_TOTAL_BV,ADI_RRB_TOTAL_C,ADI_R_ONSET_TOTAL_D,ADI_R_RSRCH_RELIABLE,ADOS_MODULE,ADOS_TOTAL,ADOS_COMM,ADOS_SOCIAL,ADOS_STEREO_BEHAV,ADOS_RSRCH_RELIABLE,ADOS_GOTHAM_SOCAFFECT,ADOS_GOTHAM_RRB,ADOS_GOTHAM_TOTAL,ADOS_GOTHAM_SEVERITY,SRS_VERSION,SRS_RAW_TOTAL,SRS_AWARENESS,SRS_COGNITION,SRS_COMMUNICATION,SRS_MOTIVATION,SRS_MANNERISMS,SCQ_TOTAL,AQ_TOTAL,COMORBIDITY,CURRENT_MED_STATUS,MEDICATION_NAME,OFF_STIMULANTS_AT_SCAN,VINELAND_RECEPTIVE_V_SCALED,VINELAND_EXPRESSIVE_V_SCALED,VINELAND_WRITTEN_V_SCALED,VINELAND_COMMUNICATION_STANDARD,VINELAND_PERSONAL_V_SCALED,VINELAND_DOMESTIC_V_SCALED,VINELAND_COMMUNITY_V_SCALED,VINELAND_DAILYLVNG_STANDARD,VINELAND_

In [185]:
# We list the paths for each subject using glob
subj_paths = sorted(glob("/content/Outputs/cpac/filt_global/rois_cc200/*.1D"))

In [189]:
# Define function which get list of unique correlations for one subject
def get_corr_list(path):

    # read the file into a DataFrame
    df = pd.read_csv(path, sep="\t")

    # calculate corr matrix for DataFrame (all rois)
    corrs = pd.DataFrame(df.corr())

    # create index with only unique combinations of row x column
    # this ensures we get an index which only gets us the "lower half"
    # of the correlation matrix
    rois_numbers = np.linspace(0, (len(corrs)-1), len(corrs), dtype = "int")
    comb_ind = [comb for comb in combinations(rois_numbers, 2)]

    # use new index to select only the unique combinations from our DataFrame
    # we save this as a dictionnary with the combination of values as keys
    # I.e., correlation of region 1 and region 4 is called "1x4"
    unique_corrs = {}
    for ind in comb_ind:
       unique_corrs[f"{ind[0]}x{ind[1]}"] = corrs.iloc[ind]

    # We add the name of the file as "ID" to the dictionnary.
    # This allows us to later identify the subjects when we create our
    # DataFrame with all our subjects
    unique_corrs["ID"] = path.split("/")[-1]

    return(unique_corrs)

In [192]:
# Create DataFrame with all subjects
list_unique_corrs = []
for path in subj_paths:
    list_unique_corrs.append(get_corr_list(path))

complete_df = pd.DataFrame(list_unique_corrs)

In [196]:
complete_df.to_csv("complete_df.csv")